In [ ]:
import pandas as pd
import altair as alt
import json, os

#local import
from utils import chart_to_html

## Data Transformation with Pandas

In [ ]:
df = pd.read_csv("Mergedcsvfile_perMeasurement.csv")
df['Zeff_stat'] = df['Zeff_stat'].apply(lambda x: float(x[1:-1]))

In [ ]:
def to_Era(run, currentYear=2017):
    if run <= 297019: return 'A'
    if run <= 299329: return 'B'
    if run <= 302029: return 'C'
    if run <= 303434: return 'D'
    if run <= 304797: return 'E'
    if run <= 306462: return 'F'
    if run <= 306826: return 'G'
    if run <= 307082: return 'H'
    return 'Unknown'

In [ ]:
# new branches
df['era'] = df['run'].apply(lambda x: to_Era(x))
df['zRec'] = df['zDel']*df['Zeff']
df.sort_values(by=['run','tdate_begin'],inplace=True)
df['measurement'] = df.groupby('run').cumcount()

In [ ]:
# sort out 
print(f"Number of measurements with NaN: {df.isnull().values.sum()}")
df = df[df.isnull() == False]

minReco=500
print(f"Number of measurements with N Z reco > {minReco}: {len(df[df['zRec']>minReco])}/{len(df)} = {round(len(df[df['zRec']>minReco])/len(df)*100,2)} %")
df = df[df['zRec'] > minReco]
df['ymin'] = df['zXSec_mc'] + df['zXSec_mc'] * df['z_relstat']
df['ymax'] = df['zXSec_mc'] - df['zXSec_mc'] * df['z_relstat']
df.to_csv('data.csv')

#df.to_json('data.json')#,open(,'w'), indent=4, sort_keys=True)

In [ ]:
df.keys()

## Plotting

Basic definitions

In [ ]:
brush = alt.selection_interval(empty='all')

selection = alt.selection_multi(fields=['era'],empty='all')


color1 = alt.condition(selection & brush,
                      alt.Color('era:N',
                      legend=None),
                      alt.value('lightgray')
                     )

color2 = alt.condition(selection,
                      alt.Color('era:N',
                      legend=None),
                      alt.value('lightgray')
                     )

#opacity = alt.condition(alt.datum.auc < selector.zRec, alt.value(0.7), alt.value(0.0))

legend = alt.Chart(df).mark_circle(size=60).encode(
    y=alt.Y('era:N', axis=alt.Axis(orient='right'), title='Era'),
    color=color2
).add_selection(
    selection
)

tooltips = [
    'fill:Q',
    'run',
    'measurement', 
    alt.X('zRec:Q',title='reconstructed Z'), 
    alt.X('GloeffB_chi2fail:Q',title='Chi2: Glo barrel fail'), 
    alt.X('GloeffB_chi2pass:Q',title='Chi2: Glo barrel pass'), 
    alt.X('GloeffE_chi2fail:Q',title='Chi2: Glo endcap fail'), 
    alt.X('GloeffE_chi2pass:Q',title='Chi2: Glo endcap pass'), 
    alt.X('SeleffB_chi2fail:Q',title='Chi2: Sel barrel fail'), 
    alt.X('SeleffB_chi2pass:Q',title='Chi2: Sel barrel pass'), 
    alt.X('SeleffE_chi2fail:Q',title='Chi2: Sel endcap fail'), 
    alt.X('SeleffE_chi2pass:Q',title='Chi2: Sel endcap pass'), 
    alt.X('HLTeffB_chi2fail:Q',title='Chi2: HLT barrel fail'), 
    alt.X('HLTeffB_chi2pass:Q',title='Chi2: HLT barrel pass'), 
    alt.X('HLTeffE_chi2fail:Q',title='Chi2: HLT endcap fail'), 
    alt.X('HLTeffE_chi2pass:Q',title='Chi2: HLT endcap pass'), 
]

chart_base = alt.Chart(df).transform_calculate(
    urlruns='../RunsAndFits/Run' + alt.datum.run+"/"
).encode(
    href='urlruns:N'
)

simple scatter plot

In [ ]:
chart_scatter_base = chart_base.encode(
    color=color2,
    tooltip=tooltips,
    opacity=alt.value(0.7),
    x=alt.X('fill', scale=alt.Scale(zero=False), title='fill'),    
)

chart_scatter_points = chart_scatter_base.mark_circle(size=60).encode(
    y=alt.Y('zXSec_mc', scale=alt.Scale(zero=False), title='fiducial cross section'),

).properties(
    width=500,
    height=250
)

chart_scatter_err = chart_scatter_base.transform_calculate(
    zXSec_mcU='parseFloat(datum.zXSec_mc) + parseFloat(datum.zXSec_mc) * parseFloat(datum.z_relstat)',
    zXSec_mcD='parseFloat(datum.zXSec_mc) - parseFloat(datum.zXSec_mc) * parseFloat(datum.z_relstat)'
).mark_line().encode(
    y='zXSec_mcU:Q',
    y2='zXSec_mcD:Q'
)

chart_scatter = alt.layer(chart_scatter_points,chart_scatter_err)

chart_scatter = chart_scatter.add_selection(
    selection
).transform_filter(
    selection 
).interactive()

chart_to_html(chart_scatter, "summary_xSec_fill")

chart_scatter | legend

simple stacked density

In [ ]:
chart_density = chart_base.transform_density(
    density='zXSec_mc',
    bandwidth=10.,
    groupby=['era'],
    extent= [25, 700],
    counts = True,
    steps=200,
    as_=["value", "density"]
).mark_area().encode(
    alt.X('value:Q', title='fiducial cross section'),
    alt.Y('density:Q', stack='zero'),
    color=color2,
    tooltip='sum(zLumi_mc)'
).properties(
    width=300, height=250
).transform_filter(
    selection 
)

chart_density | legend

Combine scatter plot and density

In [ ]:
#chart_scatter | chart_density | legend
chart_combined = alt.hconcat(chart_scatter, chart_density, legend)
chart_combined

In [ ]:
#save the plot 
chart_to_html(chart_combined, "summary_xSec_fill_pileup")

Pileup

In [ ]:
base = chart_base.transform_calculate(
    ymin='datum.zXSec_mc + datum.zXSec_mc * datum.z_relstat',
    ymax='datum.zXSec_mc - datum.zXSec_mc * datum.z_relstat'
).encode(
    color=color2,
    x=alt.X('fill', scale=alt.Scale(zero=False), title='fill'),    
).mark_circle(size=60).encode(
    color=color1,
    x=alt.X('pileUp:Q', scale=alt.Scale(zero=False), title='Pileup'),
    tooltip=tooltips,
    opacity=alt.value(0.7)
).properties(
    width=500,
    height=250
).add_selection(
    brush
).transform_filter(
    selection 
)

chart_scatter_base = chart_base

charts = []
for key, label in (
    ('zXSec_mc', 'fiducial cross section'),
    ('zXSec', 'fiducial cross section (w/o PU correction)'),
    ('zDel_mc','Z del'), 
    ('zDel','Z del w/o correction'),
    ('ZMCeff','Z efficiency'),    
    ('Zeff','Z efficiency w/o correction'),    
    ('fr', 'Z fake rate'),
    ('GloeffE', 'Global muon endcap efficiency'),
    ('GloeffB', 'Global muon barrel efficiency'),
    ('SeleffE', 'Selection muon endcap efficiency'),
    ('SeleffB', 'Selection muon barrel efficiency'),
    ('HLTeffE', 'HLT muon endcap efficiency'),
    ('HLTeffB', 'HLT muon barrel efficiency'),
    ('ZMCeffEE','Z endcap-endcap efficiency'),
    ('ZMCeffBE','Z barrel-endcap efficiency'),
    ('ZMCeffBB','Z barrel-barrel efficiency'),
    ('zRate_mc','Z rate del)'), 
    ('zRate','Z rate del w/o correction'),
):
    chart = base.encode(
        y=alt.Y(key, scale=alt.Scale(zero=False), title=label)
    )
    chart = alt.hconcat(chart,legend)
    charts.append(chart)
    
    chart_to_html(chart, key+"_pileup")


chart = alt.vconcat(*charts)
chart_to_html(chart, "summary_pileup")
chart
#charts[0]

Combined plots

In [ ]:

color1 = alt.condition(selection & brush,
                      alt.Color('era:N',
                      legend=None),
                      alt.value('lightgray')
                     )
color2 = alt.condition(selection,
                      alt.Color('era:N',
                      legend=None),
                      alt.value('lightgray')
                     )

base = chart_base.mark_point().mark_circle(size=60).encode(
    color=color1,
    tooltip=tooltips
).properties(
    width=500,
    height=250
).add_selection(
    brush
).transform_filter(
    selection
)

legend = alt.Chart(df).mark_circle(size=60).encode(
    y=alt.Y('era:N', axis=alt.Axis(orient='right'), title='Era'),
    color=color2
).add_selection(
    selection
)

chart11 = base.encode(
    x=alt.X('fill:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('zXSec_mc:Q', title='fiducial cross section')
)
chart12 = base.encode(
    x=alt.X('pileUp:Q', scale=alt.Scale(zero=False), title='Pileup'),
    y=alt.Y('ZMCeff:Q', scale=alt.Scale(zero=False), title='Z efficiency')
)
chart21 = base.encode(
    x=alt.X('zLumi_mc:Q', scale=alt.Scale(zero=False), title='Z lumi'),
    y=alt.Y('lumiRec:Q', scale=alt.Scale(zero=False), title='reference lumi')
)
chart22 = base.encode(
    x=alt.X('zDel_mc:Q', scale=alt.Scale(zero=False), title='N Z del'),
    y=alt.Y('zRate_mc:Q', scale=alt.Scale(zero=False), title='Z rate del')
)

chart1 = alt.hconcat(chart11, chart12, legend)
chart2 = alt.hconcat(chart21, chart22)
chart = alt.vconcat(chart1, chart2)
#chart.save('chart_summary.html',embed_options={'renderer':'svg'})
chart_to_html(chart, "summary")
chart
